<a href="https://colab.research.google.com/github/adlihs/instatXML/blob/main/Instat_XML_file_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Process to work with Instat's xml file

## Import libraries

In [1]:
!pip install pandas_read_xml
!pip install -U pandasql
import pandas_read_xml as pdx 
from pandas_read_xml import flatten, fully_flatten, auto_separate_tables
import os,glob 
import pandas as pd
import time
import numpy as np
from pandasql import sqldf 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 461 kB 7.9 MB/s 
     |████████████████████████████████| 138 kB 58.2 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple

## Setup the name of the tournament

In [2]:
nombre_torneo = "La Liga"

## Setup the google drive folder's path
The idea is to download all the xml from Instat and put all in a google drive folder.

In [10]:
 #this is an example of my folder's path
 %cd /content/drive/MyDrive/testXML

/content/drive/MyDrive/testXML


## Remove excel files from previous process
The script will generate an excel file for each xml, so the idea is to remove the files from previous process.

In [11]:
print('Finding and removing old excel files...')
excel_files = []
for file in glob.glob("*.xlsx"):
    excel_files.append(file)

for file in excel_files:
  os.remove(file)

print('done!')

Finding and removing old excel files...
done!


## Let's find all the XML files in the folder

In [12]:
print('Finding xml files...')
xml_files = []
for file in glob.glob("*.xml"):
    xml_files.append(file)

print('done!')

Finding xml files...
done!


## Generate an excel file for each xml file

In [13]:
print('Creating excel files from xmls')
open_file =""
for xml in xml_files:
  # This line will remove the name of the competion from the xml file name
  file_name = xml.replace('--Spain--Primera-Division','') # CHOOSE THE TEXT BETWEEN THE DATE AND THE FIRST TEAM NAME IN FILE NAME
  # This line will remove the text '.xml' from the variable 'file_name' 
  file_name = file_name.replace('.xml','')
  # This line will create a dataframe with the xml data
  open_file = pdx.read_xml(xml, ['file', 'ALL_INSTANCES', 'instance'], root_is_rows=False)
  # This line is to create a new column named 'file_name', the column will contain the information of the game
  open_file["file_name"] = file_name
  # This line will generate an excel file
  open_file.to_excel(file_name + ".xlsx")
  #print(open_file.file_name)

print('done!')
open_file

Creating excel files from xmls
done!


,ID,start,end,code,label,pos_x,pos_y,file_name
0,1,2.29,22.29,10. Iker Muniain,"[{'group': 'Team', 'text': 'Athletic Bilbao'},...",51.8,34.6,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...
1,2,2.29,22.29,10. Iker Muniain,"[{'group': 'Team', 'text': 'Athletic Bilbao'},...",51.8,34.6,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...
2,3,4.61,24.61,4. Inigo Martinez,"[{'group': 'Team', 'text': 'Athletic Bilbao'},...",33.4,45.4,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...
3,4,4.61,24.61,4. Inigo Martinez,"[{'group': 'Team', 'text': 'Athletic Bilbao'},...",33.4,45.4,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...
4,5,5.88,25.88,6. Mikel Vesga Arruti,"[{'group': 'Team', 'text': 'Athletic Bilbao'},...",44.9,45.9,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...
...,...,...,...,...,...,...,...,...
3976,3977,5976.55,5996.55,10. Iker Muniain,"[{'group': 'Team', 'text': 'Athletic Bilbao'},...",69.6,24.2,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...
3977,3978,5976.55,5996.55,10. Iker Muniain,"[{'group': 'Team', 'text': 'Athletic Bilbao'},...",69.6,24.2,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...
3978,3979,5976.55,5996.55,10. Iker Muniain,"[{'group': 'Team', 'text': 'Athletic Bilbao'},...",69.6,24.2,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...
3979,3980,5977.98,5997.98,1. Marko Dmitrovic,"[{'group': 'Team', 'text': 'Sevilla'}, {'group...",5.5,34.5,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...


## Merge all excel files in one dataframe

In [14]:
print('Merging files...')
all_data = pd.DataFrame()
for f in glob.glob("*.xlsx"):
    df = pd.read_excel(f)
    all_data = all_data.append(df,ignore_index=True)
print('done!')
all_data

Merging files...
done!


,Unnamed: 0,ID,start,end,code,label,pos_x,pos_y,file_name
0,0,1,2.29,22.29,10. Iker Muniain,"[{'group': 'Team', 'text': 'Athletic Bilbao'},...",51.8,34.6,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...
1,1,2,2.29,22.29,10. Iker Muniain,"[{'group': 'Team', 'text': 'Athletic Bilbao'},...",51.8,34.6,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...
2,2,3,4.61,24.61,4. Inigo Martinez,"[{'group': 'Team', 'text': 'Athletic Bilbao'},...",33.4,45.4,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...
3,3,4,4.61,24.61,4. Inigo Martinez,"[{'group': 'Team', 'text': 'Athletic Bilbao'},...",33.4,45.4,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...
4,4,5,5.88,25.88,6. Mikel Vesga Arruti,"[{'group': 'Team', 'text': 'Athletic Bilbao'},...",44.9,45.9,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...
...,...,...,...,...,...,...,...,...,...
3976,3976,3977,5976.55,5996.55,10. Iker Muniain,"[{'group': 'Team', 'text': 'Athletic Bilbao'},...",69.6,24.2,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...
3977,3977,3978,5976.55,5996.55,10. Iker Muniain,"[{'group': 'Team', 'text': 'Athletic Bilbao'},...",69.6,24.2,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...
3978,3978,3979,5976.55,5996.55,10. Iker Muniain,"[{'group': 'Team', 'text': 'Athletic Bilbao'},...",69.6,24.2,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...
3979,3979,3980,5977.98,5997.98,1. Marko Dmitrovic,"[{'group': 'Team', 'text': 'Sevilla'}, {'group...",5.5,34.5,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...


## Split column `'Label'` by comma `','`

In [15]:
#Create new dataframe with the new columns from the column 'LABEL'
print('Creating new data frame')
all_data2 = all_data['label'].str.split(',', expand=True)


#Merge original dataframe and new dataframe
print('Merging dataframes')
all_data = pd.concat([all_data, all_data2], axis=1, sort=False)

###REMOVE UNNCESSARY COLUMNS
print('Removing columns')
all_data.drop('label', inplace=True, axis=1)
all_data.drop(0, inplace=True, axis=1)
all_data.drop(2, inplace=True, axis=1)
all_data.drop(4, inplace=True, axis=1)

#### RENAME NEW COLUMNS (Teams, Actions, Time)
print('Reanaming columns')
all_data = all_data.rename(columns={1: "Equipos", 3: "Acciones",5:"Tiempo"})


#### REPLACE CHARACTERS IN COLUMNS EQUIPOS, ACCIONES, TIEMPO
print('Replacing special characters')
import re
chars_to_remove = ['.', '-', '(', ')',']','\'', '','}',':']
regular_expression = '[' + re.escape (''. join (chars_to_remove)) + ']'

all_data['Equipos'] = all_data['Equipos'].str.replace(regular_expression, '', regex=True)
all_data['Acciones'] = all_data['Acciones'].str.replace(regular_expression, '', regex=True)
all_data['Tiempo'] = all_data['Tiempo'].str.replace(regular_expression, '', regex=True)

all_data['Equipos'] = all_data['Equipos'].str.replace('text','')
all_data['Acciones'] = all_data['Acciones'].str.replace('text','')
all_data['Tiempo'] = all_data['Tiempo'].str.replace('text','')


print('done!')


Creating new data frame
Merging dataframes
Removing columns
Reanaming columns
Replacing special characters
done!


## Preview the dataframe

In [16]:
all_data.head(20)

,Unnamed: 0,ID,start,end,code,pos_x,pos_y,file_name,Equipos,Acciones,Tiempo
0,0,1,2.29,22.29,10. Iker Muniain,51.8,34.6,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...,Athletic Bilbao,Passes accurate,1st half
1,1,2,2.29,22.29,10. Iker Muniain,51.8,34.6,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...,Athletic Bilbao,Positional attacks,1st half
2,2,3,4.61,24.61,4. Inigo Martinez,33.4,45.4,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...,Athletic Bilbao,Positional attacks,1st half
3,3,4,4.61,24.61,4. Inigo Martinez,33.4,45.4,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...,Athletic Bilbao,Passes accurate,1st half
4,4,5,5.88,25.88,6. Mikel Vesga Arruti,44.9,45.9,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...,Athletic Bilbao,Positional attacks,1st half
5,5,6,5.88,25.88,6. Mikel Vesga Arruti,44.9,45.9,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...,Athletic Bilbao,Passes accurate,1st half
6,6,7,7.49,27.49,4. Inigo Martinez,30.1,46.7,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...,Athletic Bilbao,Positional attacks,1st half
7,7,8,7.49,27.49,4. Inigo Martinez,30.1,46.7,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...,Athletic Bilbao,Passes accurate,1st half
8,8,9,10.06,30.06,12. Daniel Vivian Moreno,27.3,24.3,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...,Athletic Bilbao,Positional attacks,1st half
9,9,10,10.06,30.06,12. Daniel Vivian Moreno,27.3,24.3,22-05-2022Sevilla-1-0-Athletic-Bilbao20220629-...,Athletic Bilbao,Passes accurate,1st half


## Get new columns from `file_name` column

In [17]:
#New column 'fecha_partido' (game_date)
print('creating new column fecha_partido')
all_data['fecha_partido'] = all_data['file_name'].str[:10]


#New column 'nombre_torneo' (tournament_name)
print('creating new column nombre_torneo')
all_data['nombre_torneo'] = nombre_torneo

#remove game date from string
print('Removing game date from string')
all_data['file_name'] = all_data['file_name'].str[10:]


#Remove last 20 characters
print('removing unnecessary last 22 last characters')
all_data['file_name'] = all_data['file_name'].str[:-21]

####RENAME COLUMN FILE_NAME
print('Reanaming column')
all_data = all_data.rename(columns={'file_name': "Partido"})

print('done!')

creating new column fecha_partido
creating new column nombre_torneo
Removing game date from string
removing unnecessary last 22 last characters
Reanaming column
done!


## Preview the dataframe with the new columns

In [18]:
all_data.head(20)

,Unnamed: 0,ID,start,end,code,pos_x,pos_y,Partido,Equipos,Acciones,Tiempo,fecha_partido,nombre_torneo
0,0,1,2.29,22.29,10. Iker Muniain,51.8,34.6,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Passes accurate,1st half,22-05-2022,La Liga
1,1,2,2.29,22.29,10. Iker Muniain,51.8,34.6,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Positional attacks,1st half,22-05-2022,La Liga
2,2,3,4.61,24.61,4. Inigo Martinez,33.4,45.4,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Positional attacks,1st half,22-05-2022,La Liga
3,3,4,4.61,24.61,4. Inigo Martinez,33.4,45.4,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Passes accurate,1st half,22-05-2022,La Liga
4,4,5,5.88,25.88,6. Mikel Vesga Arruti,44.9,45.9,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Positional attacks,1st half,22-05-2022,La Liga
5,5,6,5.88,25.88,6. Mikel Vesga Arruti,44.9,45.9,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Passes accurate,1st half,22-05-2022,La Liga
6,6,7,7.49,27.49,4. Inigo Martinez,30.1,46.7,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Positional attacks,1st half,22-05-2022,La Liga
7,7,8,7.49,27.49,4. Inigo Martinez,30.1,46.7,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Passes accurate,1st half,22-05-2022,La Liga
8,8,9,10.06,30.06,12. Daniel Vivian Moreno,27.3,24.3,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Positional attacks,1st half,22-05-2022,La Liga
9,9,10,10.06,30.06,12. Daniel Vivian Moreno,27.3,24.3,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Passes accurate,1st half,22-05-2022,La Liga


## Remove white spaces from the column `Acciones` (actions)

In [19]:
#TRIM spaces
all_data['Acciones'] = all_data['Acciones'].str.strip()
all_data['Equipos'] = all_data['Equipos'].str.strip()

## Setup columns with the `home team name` and the `away team name`

In [20]:
# Replace score in game description with @
all_data["partido_text"] = all_data["Partido"].replace(to_replace=r'[-]+\d[-]+\d+[-]', value='@', regex=True)

# Create a temp dataframe with columns 'home_team'/'away_team'
localia_df = all_data['partido_text'].str.split('@', expand=True)

# Merge new temp dataframe with all_data dataframe
all_data = pd.concat([all_data, localia_df], axis=1, sort=False)

#Rename new columns
all_data.rename(columns={0: 'equipo_casa', 1: 'equipo_visita'}, inplace=True)

#Drop column 'partido_text'
all_data.drop('partido_text', inplace=True, axis=1)

## Preview the dataframe with the new changes

In [21]:
all_data.head(20)

,Unnamed: 0,ID,start,end,code,pos_x,pos_y,Partido,Equipos,Acciones,Tiempo,fecha_partido,nombre_torneo,equipo_casa,equipo_visita
0,0,1,2.29,22.29,10. Iker Muniain,51.8,34.6,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Passes accurate,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba
1,1,2,2.29,22.29,10. Iker Muniain,51.8,34.6,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Positional attacks,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba
2,2,3,4.61,24.61,4. Inigo Martinez,33.4,45.4,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Positional attacks,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba
3,3,4,4.61,24.61,4. Inigo Martinez,33.4,45.4,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Passes accurate,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba
4,4,5,5.88,25.88,6. Mikel Vesga Arruti,44.9,45.9,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Positional attacks,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba
5,5,6,5.88,25.88,6. Mikel Vesga Arruti,44.9,45.9,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Passes accurate,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba
6,6,7,7.49,27.49,4. Inigo Martinez,30.1,46.7,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Positional attacks,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba
7,7,8,7.49,27.49,4. Inigo Martinez,30.1,46.7,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Passes accurate,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba
8,8,9,10.06,30.06,12. Daniel Vivian Moreno,27.3,24.3,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Positional attacks,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba
9,9,10,10.06,30.06,12. Daniel Vivian Moreno,27.3,24.3,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Passes accurate,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba


## Get columns for: **`Score`**, **`Home and Away goals`**

In [22]:
# Get the score from column 'Partido' (game)
all_data["marcador"] = all_data['Partido'].str.findall(r'\d[-]+\d+')

#convert column 'marcador' to string
all_data["marcador"] = all_data["marcador"].astype('str') 

# marcador en formato string
all_data["marcador"] = all_data["marcador"].str.slice(start=2, stop=-2)


#New column with the home team goals ('goles_local')
all_data['goles_local'] = all_data["marcador"].str.slice(start=0, stop=1)

#New column with the away team goals ('goles_visita')
all_data['goles_visita'] = all_data["marcador"].str.slice(start=2, stop=3)




## Preview the dataframe with the changes

In [23]:
all_data.head(20)

,Unnamed: 0,ID,start,end,code,pos_x,pos_y,Partido,Equipos,Acciones,Tiempo,fecha_partido,nombre_torneo,equipo_casa,equipo_visita,marcador,goles_local,goles_visita
0,0,1,2.29,22.29,10. Iker Muniain,51.8,34.6,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Passes accurate,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba,1-0,1,0
1,1,2,2.29,22.29,10. Iker Muniain,51.8,34.6,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Positional attacks,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba,1-0,1,0
2,2,3,4.61,24.61,4. Inigo Martinez,33.4,45.4,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Positional attacks,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba,1-0,1,0
3,3,4,4.61,24.61,4. Inigo Martinez,33.4,45.4,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Passes accurate,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba,1-0,1,0
4,4,5,5.88,25.88,6. Mikel Vesga Arruti,44.9,45.9,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Positional attacks,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba,1-0,1,0
5,5,6,5.88,25.88,6. Mikel Vesga Arruti,44.9,45.9,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Passes accurate,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba,1-0,1,0
6,6,7,7.49,27.49,4. Inigo Martinez,30.1,46.7,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Positional attacks,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba,1-0,1,0
7,7,8,7.49,27.49,4. Inigo Martinez,30.1,46.7,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Passes accurate,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba,1-0,1,0
8,8,9,10.06,30.06,12. Daniel Vivian Moreno,27.3,24.3,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Positional attacks,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba,1-0,1,0
9,9,10,10.06,30.06,12. Daniel Vivian Moreno,27.3,24.3,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Passes accurate,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba,1-0,1,0


## Setup a column with the team that "suffers" the action


In [24]:
# create a list of our conditions

conditions = [
    (all_data["Equipos"] == all_data["equipo_casa"]),
    (all_data["Equipos"] == all_data["equipo_visita"])]

# create a list of the values we want to assign for each condition
values = [all_data["equipo_visita"], all_data["equipo_casa"]]

# create a new column and use np.select to assign values to it using our lists as arguments
all_data["rival"] = np.select(conditions, values)


## Preview the changes

In [25]:
all_data.head(30)

,Unnamed: 0,ID,start,end,code,pos_x,pos_y,Partido,Equipos,Acciones,Tiempo,fecha_partido,nombre_torneo,equipo_casa,equipo_visita,marcador,goles_local,goles_visita,rival
0,0,1,2.29,22.29,10. Iker Muniain,51.8,34.6,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Passes accurate,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba,1-0,1,0,0
1,1,2,2.29,22.29,10. Iker Muniain,51.8,34.6,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Positional attacks,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba,1-0,1,0,0
2,2,3,4.61,24.61,4. Inigo Martinez,33.4,45.4,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Positional attacks,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba,1-0,1,0,0
3,3,4,4.61,24.61,4. Inigo Martinez,33.4,45.4,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Passes accurate,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba,1-0,1,0,0
4,4,5,5.88,25.88,6. Mikel Vesga Arruti,44.9,45.9,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Positional attacks,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba,1-0,1,0,0
5,5,6,5.88,25.88,6. Mikel Vesga Arruti,44.9,45.9,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Passes accurate,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba,1-0,1,0,0
6,6,7,7.49,27.49,4. Inigo Martinez,30.1,46.7,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Positional attacks,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba,1-0,1,0,0
7,7,8,7.49,27.49,4. Inigo Martinez,30.1,46.7,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Passes accurate,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba,1-0,1,0,0
8,8,9,10.06,30.06,12. Daniel Vivian Moreno,27.3,24.3,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Positional attacks,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba,1-0,1,0,0
9,9,10,10.06,30.06,12. Daniel Vivian Moreno,27.3,24.3,Sevilla-1-0-Athletic-Bilba,Athletic Bilbao,Passes accurate,1st half,22-05-2022,La Liga,Sevilla,Athletic-Bilba,1-0,1,0,0


## Remove unnecesary columns

In [26]:
all_data.drop(all_data.columns[0], axis=1,inplace=True)

## Export the consolidated dataframe to Excel file

In [27]:

print('Creating excel file with the dataframe all_data')
all_data.to_excel('UNAFUT_'+nombre_torneo+'.xlsx') # YOU CAN SETUP THE FILE NAME THAT YOU WANT
print('done!')

Creating excel file with the dataframe all_data
done!


## Delete all excel files except the file created in the previous step

In [28]:
print('Detecting all .xlsx files')
excel_files = []
for file in glob.glob("*.xlsx"):
    excel_files.append(file)

print('Removing all excel files except UNAFUT_' + nombre_torneo +'.xlsx')
for file in excel_files:
  if file != ('UNAFUT_' + nombre_torneo + '.xlsx') : # HERE CHANGE AND PUT THE FILE NAME YOU PUT IN THE PREVIOUS STEP
    os.remove(file)

print('All process is done!')

Detecting all .xlsx files
Removing all excel files except UNAFUT_La Liga.xlsx
All process is done!
